# Libraries Used

In [52]:
from __future__ import print_function
from __future__ import division

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization

# Load Dummy Data
* 1,000 rows and 45 features

In [53]:
data, target = make_classification(

    n_samples = 1000,
    n_features = 45,
    n_informative = 12,
    n_redundant = 7
)

In [54]:
data.shape

(1000, 45)

# Functions to Optimize

Optimize f1 value. Best at 1 and worst at 0

In [55]:
# Support Vector Classification 
def svccv(C, gamma):
    val = cross_val_score(
        SVC(C=C, gamma=gamma, random_state=2),
        data,
        target,
        scoring = 'f1',
        cv = 2
    ).mean()
    
    return val

In [56]:
# Random Forest Classifier
def rfccv(n_estimators, min_samples_split, max_features):
    val = cross_val_score(
        RFC(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999),
            random_state=2
        ),
        data, target, scoring= 'f1', cv=2
    ).mean()
    return val

In [57]:
svcBO = BayesianOptimization(
    svccv,
    {
      'C': (0.001, 100),
      'gamma': (0.0001, 0.1)
    }
)

svcBO.explore({'C': [0.001, 0.01, 0.1], 'gamma': [0.001, 0.01, 0.1]})


In [58]:
svcBO.maximize(n_iter=10)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    1 | 00m00s |    0.00000 |    0.0010 |    0.0010 | 
    2 | 00m00s |    0.00000 |    0.0100 |    0.0100 | 
    3 | 00m00s |    0.00000 |    0.1000 |    0.1000 | 
    4 | 00m00s |    0.93168 |   74.2354 |    0.0253 | 
    5 | 00m00s |    0.93063 |    6.4498 |    0.0290 | 
    6 | 00m00s |    0.93400 |   94.3772 |    0.0065 | 
    7 | 00m00s |    0.85812 |   66.9093 |    0.0630 | 
    8 | 00m00s |    0.91434 |    6.1121 |    0.0509 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    9 | 00m05s |    0.83009 |   47.9436 |    0.0001 | 
   10 | 00m04s |    0.82680 |   31.2205 |    0.0001 | 
   11 | 00m05s |    0.24988 |   84.9184 |    0.1000 | 
   12 | 00m07s |    0.81876 |   20.5179 |    0.0001 | 
   13 | 00m05s |    0.24988 |  100.0000 |    0.1000 | 
   14 | 00m05s |    0.24988 | 

In [59]:
print('max params is: ', svcBO.res['max']['max_params'])
print('max params gives max val of: ', svcBO.res['max']['max_val'])

max params is:  {'C': 94.377150800433597, 'gamma': 0.0064587736691996125}
max params gives max val of:  0.934


In [60]:
rfcBO = BayesianOptimization(
        rfccv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999)}
    )

In [61]:
rfcBO.maximize(n_iter=10)

Initialization
-------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_features |   min_samples_split |   n_estimators | 
    1 | 00m00s |    0.85312 |         0.1079 |             17.0486 |        80.3507 | 
    2 | 00m00s |    0.84824 |         0.7149 |             19.9874 |        41.1463 | 
    3 | 00m01s |    0.86268 |         0.5547 |              9.7181 |        75.5495 | 
    4 | 00m03s |    0.86264 |         0.8445 |              5.0469 |       171.9895 | 
    5 | 00m00s |    0.86306 |         0.1554 |             23.2333 |       123.8864 | 
Bayesian Optimization
-------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_features |   min_samples_split |   n_estimators | 
    6 | 00m11s |    0.84309 |         0.8559 |             25.0000 |       250.0000 | 
    7 | 00m06s |    0.80693 |         0.1963 |              2.0589 |        10.3012 | 
    8 | 

In [62]:
print('max params is: ', rfcBO.res['max']['max_params'])
print('max params gives max val of: ', rfcBO.res['max']['max_val'])

max params is:  {'n_estimators': 93.227932742371948, 'min_samples_split': 2.011549360233869, 'max_features': 0.20106320259453903}
max params gives max val of:  0.87950871834
